In [ ]:
import os
import os.path as op
import sys

import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import pandas as pd

#from bluemath_tk.wrappers.swash.swash_wrapper import SwashModelWrapper
from IPython.display import HTML

from scripts.wrappers import SwashModelWrapper_friction

from scripts.bathymetry import reef_profile
from scripts.plots import plot_case_config

## 01.Define Case Variables

#### Necessary Paths

In [ ]:
templates_dir = '/workspaces/ONDAS_Swash/templates/templates_friction'
output_dir = '/workspaces/ONDAS_Swash/cases/friction'

#### Hydrodynamic Forcing

In [ ]:
Hs = 2.5
Tp = 10
Hs_L0 =  round((2 * np.pi * (Hs)) / (9.81 * (Tp ** 2)),4)  # Should be between 0.005 and 0.05
WL = 0.0

In [ ]:
Hs_L0

In [ ]:
dx = 1

h0 = 15                     # offshore depth (m)
Slope1 = 0.05               # fore shore slope
Slope2 = 0.1                # inner shore slope
Wreef = 200                 # reef bed width (m)
Wfore = 500                 # flume length before fore toe (m)
bCrest = 10                 # beach heigh (m)
emsl = 2.5                  # mean sea level (m)

x_profile, depth_array = reef_profile(dx, h0, Slope1, Slope2, Wreef, Wfore, bCrest, emsl)

In [ ]:
plot_case_config(x=x_profile, z=depth_array, wave_height=Hs, wave_period = Tp, WL=WL)

In [ ]:
params = pd.DataFrame()
params['friction_coef'] = np.round(np.arange(0.1,0.35,0.05),3) # Vector de 8 a 20 de 1 en 1

params['Hs'] = Hs
params['Hs_L0'] = Hs_L0 
params['WL'] = WL

params['pos_ini'] = 800
params['pos_fin'] = 1000

In [ ]:
params

In [ ]:
metamodel_parameters = params.to_dict(orient='list')
fixed_parameters = {
    "dxinp": 1,  # bathymetry grid spacing  # Friction manning coefficient (m^-1/3 s)
    "comptime": 1600,  # Simulation duration (s)
    "warmup": 300,  # Warmup duration (s)
    "n_nodes_per_wavelength": 60,  # number of nodes per wavelength
}

swash_model = SwashModelWrapper_friction(
    templates_dir=templates_dir,
    metamodel_parameters=metamodel_parameters,
    fixed_parameters=fixed_parameters,
    output_dir=output_dir,
    depth_array=-depth_array,
)

In [ ]:
swash_model.build_cases()

In [ ]:
swash_model.run_cases(launcher='serial')

In [ ]:
vars_to_postprocess = ["Msetup", "Hrms", "Hfreqs" , "Ru2"]
postprocessed = swash_model.postprocess_cases(output_vars=vars_to_postprocess)

In [ ]:
postprocessed

In [ ]:
params

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.cm as cm

# Number of cases
n_cases = len(postprocessed.case_num.values)

# Assign a unique color per case

if n_cases <= 10:
    cmap = plt.cm.get_cmap("tab10", n_cases)
else:
    cmap = plt.cm.get_cmap("tab20", n_cases)

colors = [cmap(i) for i in range(n_cases)]
# Create figure with 5 subplots (one per variable)
fig, axs = plt.subplots(5, 1, figsize=(10, 16), sharex=True)

ax_Hs, ax_Hss, ax_Hrms, ax_IG, ax_Setup = axs

for i in range(n_cases):
    post_process_case = postprocessed.isel(case_num=i)
    Xp = post_process_case.Xp
    color = colors[i]
    fc = params['friction_coef'][i]

    # --- Hs --------------------------------------------------------
    ax_Hs.plot(Xp, post_process_case.Hs,
               label=f"Case {i+1} (fc={fc})", lw=2, color=color)

    # --- Hss -------------------------------------------------------
    ax_Hss.plot(Xp, post_process_case.Hss,
                label=f"Case {i+1} (fc={fc})", lw=2, color=color)

    # --- Hrms ------------------------------------------------------
    ax_Hrms.plot(Xp, post_process_case.Hrms,
                 label=f"Case {i+1} (fc={fc})", lw=2, color=color)

    # --- IG --------------------------------------------------------
    ax_IG.plot(Xp, post_process_case.ig,
               label=f"Case {i+1} (fc={fc})", lw=2, color=color)

    # --- Mean Setup ------------------------------------------------
    ax_Setup.plot(Xp, post_process_case.Msetup,
                  label=f"Case {i+1} (fc={fc})", lw=2, color=color)


# ---- Axis labels ----------------------------------------------------
ax_Hs.set_ylabel("Hs (m)")
ax_Hss.set_ylabel("Hss (m)")
ax_Hrms.set_ylabel("Hrms (m)")
ax_IG.set_ylabel("IG (m)")
ax_Setup.set_ylabel("Mean Setup (m)")
ax_Setup.set_xlabel("Cross-shore position Xp (m)")
ax_Setup.set_ylim(0,0.5)
# ---- Formatting -----------------------------------------------------
for ax in axs:
    ax.grid(ls="--", alpha=0.4)

# One legend for all panels
ax_Hs.legend(ncol=5, bbox_to_anchor=(0.5, 1.2),
             loc="upper center", frameon=False)

fig.suptitle("Wave Transformation Across Cases", fontsize=16, y=0.98)

plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()


In [ ]:
# soft, pleasant colors
line_color = "#7fb3d5"
marker_edge = "#5dade2"

x_vals = params["friction_coef"]
y_vals = postprocessed.Ru2

plt.figure(figsize=(8, 4))

plt.plot(
    x_vals,
    y_vals,
    color=line_color,
    linewidth=3,
    marker="o",
    markersize=8,
    markerfacecolor="white",
    markeredgecolor=marker_edge,
    markeredgewidth=2
)

plt.title("Ru2 vs Friction Coefficient", fontsize=16)
plt.xlabel("Friction Coefficient", fontsize=12)
plt.ylabel("Ru2 Value", fontsize=12)

plt.grid(alpha=0.25)

# soften the spines for a cleaner look
ax = plt.gca()
for spine in ax.spines.values():
    spine.set_linewidth(1.1)
    spine.set_color("#b4cbe6")

plt.tight_layout()
plt.show()